In [1]:
from data_gen_utils import *
import sys
sys.path.append("../") # go to parent dir

import numpy as np
import torch
import matplotlib.pyplot as plt
import mpmath

from synthetic.generate import SingleTaskTreeDepsGenerator
from metal.label_model import LabelModel, CliqueTree
from metal.label_model.utils import (
    compute_mu,
    compute_covariance,
    compute_inv_covariance,
    print_matrix,
    visualize_matrix
)

/lfs/1/annhe/anaconda3/envs/metal/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
M = 6
n = 10000
K = 1
dg = DataGenerator(M,n)

In [3]:
dg.generate_O(k=3)

In [4]:
print(dg.deps)

[(0, 1), (2, 3), (4, 5)]


In [5]:
# remove deps
print(dg.deps_no_diag)

[(0, 1), (2, 3), (4, 5)]


In [6]:
print(dg.L)

w,h = (dg.L).shape
L = np.zeros((w,h),dtype=float)
for i in range(w):
    for j in range(h):
        if dg.L[i,j] > 0:
            L[i,j] = 1
        else:
            L[i,j] = 0

[[-1.  1. -1. ...  1.  1. -1.]
 [ 1.  1. -1. ...  1. -1. -1.]
 [ 1.  1.  1. ...  1.  1. -1.]
 [ 1.  1.  1. ... -1. -1. -1.]
 [-1.  1.  1. ... -1. -1. -1.]
 [-1.  1.  1. ... -1. -1. -1.]]


In [7]:
print(L)
print(dg.deps_no_diag)

[[0. 1. 0. ... 1. 1. 0.]
 [1. 1. 0. ... 1. 0. 0.]
 [1. 1. 1. ... 1. 1. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]]
[(0, 1), (2, 3), (4, 5)]


In [8]:
lm = LabelModel(k=1)
lm.config['higher_order_cliques'] = False
lm.config['verbose'] = True

In [ ]:
lm.train(L=dg.L.T,
         deps=dg.deps_no_diag,
         all_unary_cliques=True,
        higher_order_cliques=True,
        n_epochs=50000,
        print_every=5000,
        lr=0.0001,
        l2=0,
        O_inv_prec=1024,)

I GOT HERE
I GOT HERE 2
I GOT HERE 3
INIT 1
GET CLIQUE TREE 1
GET CLIQUE TREE 2
GET CLIQUE TREE 3
GET CLIQUE TREE 4
GET CLIQUE TREE 5
INIT 2
CLIQUE TREE 1
START INDEX:  9
I GOT HERE 4
L AUG SAHPE:  (10000, 9)
ALL UNARY CLIQUES
HIGHER ORDER CLIQUES
(9, 9)
[[0.4956 0.2772 0.2858 0.2136 0.3181 0.2395 0.2227 0.2772 0.1826]
 [0.2772 0.3097 0.2081 0.1815 0.2509 0.2024 0.1883 0.2772 0.157 ]
 [0.2858 0.2081 0.4974 0.2509 0.3003 0.2183 0.204  0.1854 0.2509]
 [0.2136 0.1815 0.2509 0.2919 0.2368 0.1939 0.1808 0.1626 0.2509]
 [0.3181 0.2509 0.3003 0.2368 0.4979 0.3038 0.3038 0.2254 0.2043]
 [0.2395 0.2024 0.2183 0.1939 0.3038 0.3263 0.3038 0.1815 0.1668]
 [0.2227 0.1883 0.204  0.1808 0.3038 0.3038 0.3038 0.1686 0.1558]
 [0.2772 0.2772 0.1854 0.1626 0.2254 0.1815 0.1686 0.2772 0.1403]
 [0.1826 0.157  0.2509 0.2509 0.2043 0.1668 0.1558 0.1403 0.2509]]
Computing O^{-1}...
L AUG SAHPE:  (10000, 9)
ALL UNARY CLIQUES
HIGHER ORDER CLIQUES
O unnorm   [4956.0  2772.0  2858.0  2136.0  3181.0  2395.0  2227.0

In [ ]:
lm._set_constants(dg.L.T)
#lm._set_dependencies(dg.deps_no_diag)
mu = compute_mu(lm._get_augmented_label_matrix(dg.L.T), dg.Y.T, K, np.full(K, 1/K))


# Test against the true parameter values
mu_est = lm.mu.detach().numpy()
print(mu_est)
print(f"Average absolute error: {np.mean(np.abs(mu_est - mu))}")

mu_est_sm = mu_est[:6]
mu_true = dg.mu
print(f"Average absolute error: {np.mean(np.abs(mu_est_sm - mu_true))}")

In [ ]:
lm.train(L=L.T,
         deps=dg.deps_no_diag,
         all_unary_cliques=True,
        higher_order_cliques=True,
        n_epochs=50000,
        print_every=5000,
        lr=0.0001,
        l2=0,
        O_inv_prec=1024,)

In [ ]:
lm._set_constants(L.T)
#lm._set_dependencies(dg.deps_no_diag)
Y = np.zeros(dg.Y.shape,dtype=float)
for i in range(n):
    if dg.Y[i] > 0:
        Y[i] = 1
    else:
        Y[i] = 0
mu = compute_mu(lm._get_augmented_label_matrix(L.T), Y.T, K, np.full(K, 1/K))


# Test against the true parameter values
mu_est = lm.mu.detach().numpy()
print(mu_est)
print(f"Average absolute error: {np.mean(np.abs(mu_est - mu))}")

mu_est_sm = mu_est[:6]
mu_true = dg.mu
print(f"Average absolute error: {np.mean(np.abs(mu_est_sm - mu_true))}")